In [1]:
!pip install langgraph openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 14.8 MB/s eta 0:00:00


In [2]:
from langgraph.graph import StateGraph, END
from openai import OpenAI
import os

In [3]:
from getpass import getpass

api_key = getpass("Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = api_key

client = OpenAI()

print("✅ API key set successfully!")

Enter your OpenAI API key: ··········
✅ API key set successfully!


In [4]:
symptom_to_disease = {
    "chest pain": ["Angina", "Heart Attack"],
    "rash": ["Eczema", "Allergic Reaction"],
    "fever": ["Flu", "Infection"]
}

disease_to_department = {
    "Angina": "Cardiology",
    "Heart Attack": "Cardiology",
    "Eczema": "Dermatology",
    "Allergic Reaction": "Dermatology",
    "Flu": "General Medicine",
    "Infection": "General Medicine"
}

def map_symptoms_to_department(symptom_text: str):
    matched = []
    for symptom, diseases in symptom_to_disease.items():
        if symptom in symptom_text.lower():
            for disease in diseases:
                department = disease_to_department.get(disease, "General Medicine")
                matched.append((disease, department))
    return matched


In [13]:
def gpt4o_medical_advice(state):
    messages = state["messages"]
    user_input = messages[-1]["content"].strip().lower()

    full_convo = "\n".join([f"{m['role']}: {m['content']}" for m in messages[-5:]])  # last 5 turns

    if any(greet in user_input for greet in ["hello", "hi", "hey"]):
        reply = "👋 Hello! I'm a medical assistant. You can tell me your symptoms, and I’ll guide you to the right department."

    elif "what to do" in user_input or "what should i do" in user_input:
        reply = (
            "Of course! Based on what you've shared so far, let me assess the symptoms.\n"
            "Please hold on..."
        )
        system_prompt = (
            "You are a helpful medical assistant. The following is a conversation with a patient. "
            "Based on the recent messages, infer what condition the patient might be experiencing and "
            "suggest which medical department they should consult. Use short, helpful responses."
        )

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": full_convo}
            ],
            temperature=0.3
        )
        reply = response.choices[0].message.content

    else:
        system_prompt = (
            "You are a medical assistant. Given a symptom description from a user, "
            "infer possible causes and suggest which department to consult. "
            "Keep responses short, professional, and clear."
        )

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ],
            temperature=0.4
        )

        reply = response.choices[0].message.content

    return {
        "messages": messages + [{
            "role": "assistant",
            "content": reply
        }]
    }


In [14]:
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph

class ChatState(TypedDict):
    messages: List[Dict[str, Any]]

graph = StateGraph(ChatState)

graph.add_node("GPT-4o", gpt4o_medical_advice)

graph.set_entry_point("GPT-4o")
graph.set_finish_point("GPT-4o")

lang_graph = graph.compile()


In [15]:
def chat_interface():
    print("🩺 Medical Chatbot: Type your symptoms (or 'exit')")
    state = {"messages": []}

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("👋 Take care!")
            break
        state['messages'].append({"role": "user", "content": user_input})
        state = lang_graph.invoke(state)
        print("Bot:", state['messages'][-1]['content'])

chat_interface()


🩺 Medical Chatbot: Type your symptoms (or 'exit')
You: Hello
Bot: 👋 Hello! I'm a medical assistant. You can tell me your symptoms, and I’ll guide you to the right department.
You: I ahve head ache
Bot: Headaches can have various causes, including tension, migraines, dehydration, stress, or sinus issues. It is advisable to consult with a General Practitioner (GP) to determine the underlying cause and appropriate treatment. If the headache is severe, persistent, or accompanied by other symptoms like vision changes or nausea, seek medical attention promptly.
You: what to do
Bot: For now, you can try resting in a quiet, dark room and applying a cold or warm compress to your head. Stay hydrated by drinking water, and consider over-the-counter pain relief like ibuprofen or acetaminophen if you can take them safely. However, it's important to consult with a General Practitioner (GP) to identify the cause of your headache and receive appropriate treatment.
You: what should I eat
Bot: If you're